# Capstone Project: Battle of the Neighborhoods

## Table of Contents

* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results](#results)
* [Conclusion](#conclusion)

## Introduction <a name="introduction"></a>

The aim of this project is to compare the cities of New York, NY, USA and Toronto, ON, Canada.  Specifically by comparing neighborhoods <br>
between two cities to see which are most similar to each other. <br>
<br>
This can be done by clustering the neighborhoods in the respective cities, New York and Toronto, showing which neighborhoods are most alike <br>
and which are most dissimilar.  Such analysis could be used by businesses, such as restaurants, that see success in one city and are looking <br>
to expand to the other city.  For example, if an italian restaurant in Toronto was looking to open a new location in New York the knowledge of <br>
which neighoborhoods in New York are most similar to those in Toronto would improve the chances of the new location succeeding. <br>
<br>
By implementing machine learning methods we will measure the similarities between neighborhoods.  By doing so the most similar neighborhoods <br>
in both cities will be grouped and clearly displayed.

## Data <a name="data"></a>

In order to solve the problem previously put forward, we will need the following for New York and Toronto:
* Geographic data of the neighborhoods 
* The number of different types of venues in each neighborhood


The sources of data to be used are the following:

* https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M (A table of postal codes and neighborhoods in Toronto)
* Geospatial_Coordinates.csv (A csv of the latitude and longitude of postal codes in Toronto)
* newyork_data.json (A json file of New York neighborhood latitude and longitude data)
* Foursquare API (To obtain the number of venues in each neighborhood)

### Imports 

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

## Methodology <a name="methodology"></a>

## Analysis <a name="analysis"></a>

## Results <a name="results"></a>

## Conclusion <a name="conclusion"></a>